In [1]:
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit
from sklearn.utils import shuffle
import numpy as np
from datasets import *
from models_test import *
from models_MR import LapRVFL, LapDeepRVFL, LapEnsembleDeepRVFL, LapBRVFL, LapBDeepRVFL, LapBEnsembleDeepRVFL
from time import time

# set random seed
np.random.seed(42)

data, label, n_class = coil20()

In [2]:
def run_SS(data, label, n_class, model_class, partition, lam=1, n_layer=1, activation='sigmoid'):
    L, U, V, T = partition[0], partition[1], partition[2], partition[3]
    n_reps = 3

    n_node = 2000 # num of nodes in hidden layer
    w_range = [-1, 1] # range of random weights
    b_range = [0, 1] # range of random biases

    unlab_accs = []
    val_accs = []
    test_accs = []
    train_time = []
    
    data, label = shuffle(data, label)
    # Cross-validation split
    for i in range(n_reps):
        ss = ShuffleSplit(n_splits=4, test_size=T, random_state=42)
        for i, ss_values in enumerate(ss.split(data, label)):
            train_index, test_index = ss_values
            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = label[train_index], label[test_index]

            _labeled = []
            _validation = []

            used = set()
            # fill up labeled set with one element from each class
            present = []
            for i in range(len(X_train)):
                if len(_labeled) == L:
                    break
                if (y_train[i] not in present) and (i not in used):
                    _labeled.append(i)
                    present.append(y_train[i])
                    used.add(i)

            # fill up validation set with one element from each class
            present = []
            for i in range(len(X_train)):
                if len(_validation) == V:
                    break
                if (y_train[i] not in present) and (i not in used):
                    _validation.append(i)
                    present.append(y_train[i])
                    used.add(i)

            # remaining training samples
            remaining = [i for i in range(len(X_train)) if i not in used]
            # print(len(remaining))
            lab_missing = L - len(_labeled)
            val_missing = V - len(_validation)
            # labeled
            X_lab, X_remain, y_lab, y_remain = train_test_split(X_train[remaining], y_train[remaining], train_size=lab_missing)
            # validation, unlabelled
            X_val, X_unlab, y_val, y_unlab = train_test_split(X_remain, y_remain, train_size=val_missing)
            X_lab, y_lab = np.append(X_lab, X_train[_labeled], 0), np.append(y_lab, y_train[_labeled], 0)
            X_val, y_val = np.append(X_val, X_train[_validation], 0), np.append(y_val, y_train[_validation], 0)

            assert len(X_lab) == L
            assert len(y_lab) == L
            assert len(X_unlab) == U
            assert len(y_unlab) == U
            assert len(X_val) == V
            assert len(y_val) == V
            assert len(X_test) == T
            assert len(y_test) == T

            # predict unlabelled dataset
            if model_class in [RVFL, DeepRVFL, EnsembleDeepRVFL, LapRVFL, LapDeepRVFL, LapEnsembleDeepRVFL]:
                model = model_class(n_node, lam, w_range, b_range, n_layer, activation=activation)
            elif model_class in [BRVFL, BDeepRVFL, BEnsembleDeepRVFL, LapBRVFL, LapBDeepRVFL, LapBEnsembleDeepRVFL]:
                model = model_class(n_node, w_range, b_range, n_layer, tol=10**(-7), activation=activation)
            t = time()
            model.train(X_lab, y_lab, n_class)
            train_time.append(time() - t)
            unlab_acc, unlab_pred = model.eval(X_unlab, y_unlab)
            # print('Unlabelled Accuracy: ', unlab_acc)
            unlab_accs.append(unlab_acc)

            # retrain on y_lab and unlab prediction, eval on val set
            new_X_train, new_y_train = np.append(X_lab, X_unlab, 0), np.append(y_lab, unlab_pred, 0)
            if model_class in [RVFL, DeepRVFL, EnsembleDeepRVFL, LapRVFL, LapDeepRVFL, LapEnsembleDeepRVFL]:
                model = model_class(n_node, lam, w_range, b_range, n_layer, activation=activation)
            elif model_class in [BRVFL, BDeepRVFL, BEnsembleDeepRVFL, LapBRVFL, LapBDeepRVFL, LapBEnsembleDeepRVFL]:
                model = model_class(n_node, w_range, b_range, n_layer, tol=10**(-7), activation=activation)
            t = time()
            model.train(new_X_train, new_y_train, n_class)
            train_time.append(time() - t)
            val_acc, _ = model.eval(X_val, y_val)
            val_accs.append(val_acc)

            test_acc, _ = model.eval(X_test, y_test)
            # print('Test accuracy: ', test_acc)val_acc
            test_accs.append(test_acc)

    # find average acc
    mean_unlab_acc = np.mean(unlab_accs), np.std(unlab_accs)
    mean_val_acc = np.mean(val_accs), np.std(val_accs)
    mean_test_acc = np.mean(test_accs), np.std(test_accs)
    mean_train_time = np.mean(train_time)
    # print('Validation Accuracy', mean_val_acc)
    # print('Test accuracy: ', mean_test_acc)

    return mean_unlab_acc, mean_val_acc, mean_test_acc, mean_train_time


In [4]:
partition = [40, 1000, 40, 360]
lam = 1
n_layer = 1
activation='sigmoid'
model_class = RVFL
lams = [2**i for i in range(-6, 12, 2)] # regularization parameter, lambda

unlab = []
val = []
test = []
t = []
for i, lam in enumerate(lams):
    unlab_acc, val_acc, test_acc, train_time = run_SS(data, label, n_class, model_class, partition, lam=lam, n_layer=1, activation='sigmoid')
    unlab.append(unlab_acc)
    val.append(val_acc)
    test.append(test_acc)
    t.append(train_time)

max_index = np.argmax(val, axis=0)[0]
opt_lam = lams[max_index]
print('Unlab: ', unlab[max_index][0], u"\u00B1", unlab[max_index][1])
print('Val: ', val[max_index][0], u"\u00B1", val[max_index][1])
print('Test: ', test[max_index][0], u"\u00B1", test[max_index][1])
print('Lambda: ', opt_lam)
print('Train time: ', t[max_index])


Unlab:  0.652 ± 0.024637369989509845
Val:  0.6729166666666667 ± 0.05632710764414904
Test:  0.6752314814814815 ± 0.02815618032329325
Lambda:  0.25
Train time:  0.2524547179539998


In [22]:
print(unlab)
print(val)

[(0.6591666666666667, 0.02933664754004603), (0.6663333333333333, 0.02112397269033979), (0.64975, 0.03482126984090808), (0.6728333333333333, 0.022912272305954766), (0.6655000000000001, 0.02587952858921507), (0.6629166666666667, 0.026465255000135975), (0.6693333333333333, 0.0266312264498318), (0.6705833333333334, 0.030070634438860027), (0.6464166666666668, 0.034922433891252325)]
[(0.6770833333333334, 0.04012783392553796), (0.6729166666666666, 0.06881310235380728), (0.6208333333333333, 0.07204840194079408), (0.6833333333333332, 0.060667582410670995), (0.6625, 0.07180703308172534), (0.6541666666666667, 0.07276770498572074), (0.69375, 0.07577722283113839), (0.6604166666666668, 0.05991168268561835), (0.6562499999999999, 0.07438987946398802)]


In [5]:
# RVFL

n_node = 10 # num of nodes in hidden layer
lam = 1 # regularization parameter, lambda
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

for i, kf_values in enumerate(kf.split(X_train, y_train)):
#     print(f'train: {train_index}, val: {val_index}')
    print('Validation: {}'.format(i + 1))
    train_index, val_index = kf_values
    X_val_train, X_val_test = X_train[train_index], X_train[val_index]
    y_val_train, y_val_test = y_train[train_index], y_train[val_index]
    model = RVFL(n_node, lam, w_range, b_range)
    model.train(X_val_train, y_val_train, n_class)
    prediction = model.predict(X_val_test, True)
    acc = model.eval(X_val_test, y_val_test)
    print(f'Validation accuracy: {acc}')
    val_acc.append(acc)
    if acc >= max(val_acc):
        max_index = train_index

X_train, y_train = X_train[max_index], y_train[max_index]
model = RVFL(n_node, lam, w_range, b_range)
model.train(X_train, y_train, n_class)
prediction = model.predict(X_test, True)
acc = model.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')

Validation: 1
Validation accuracy: 0.9396551724137931
Validation: 2
Validation accuracy: 0.9310344827586207
Validation: 3
Validation accuracy: 0.9478260869565217
Validation: 4
Validation accuracy: 0.9565217391304348
Validation: 5
Validation accuracy: 0.9391304347826087
Validation: 6
Validation accuracy: 0.9217391304347826
Validation: 7
Validation accuracy: 0.9652173913043478
Validation: 8
Validation accuracy: 0.9478260869565217
Validation: 9
Validation accuracy: 0.9478260869565217
Validation: 10
Validation accuracy: 0.9565217391304348

Test accuracy: 0.9236111111111112


In [6]:
# dRVFL

n_node = 10 # num of nodes in hidden layer
n_layer = 2 # num of hidden layers
lam = 1 # regularization parameter, lambda
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

for i, kf_values in enumerate(kf.split(X_train, y_train)):
#     print(f'train: {train_index}, val: {val_index}')
    print('Validation: {}'.format(i + 1))
    train_index, val_index = kf_values
    X_val_train, X_val_test = X_train[train_index], X_train[val_index]
    y_val_train, y_val_test = y_train[train_index], y_train[val_index]
    model = DeepRVFL(n_node, lam, w_range, b_range, n_layer)
    model.train(X_val_train, y_val_train, n_class)
    prediction = model.predict(X_val_test, False)
    acc = model.eval(X_val_test, y_val_test)
    print(f'Validation accuracy: {acc}')
    val_acc.append(acc)
    if acc >= max(val_acc):
        max_index = train_index

X_train, y_train = X_train[max_index], y_train[max_index]
model = DeepRVFL(n_node, lam, w_range, b_range, n_layer)
model.train(X_train, y_train, n_class)
prediction = model.predict(X_test, False)
acc = model.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')

Validation: 1
Validation accuracy: 0.9396551724137931
Validation: 2
Validation accuracy: 0.9482758620689655
Validation: 3
Validation accuracy: 0.9391304347826087
Validation: 4
Validation accuracy: 0.9652173913043478
Validation: 5
Validation accuracy: 0.9304347826086956
Validation: 6
Validation accuracy: 0.9217391304347826
Validation: 7
Validation accuracy: 0.9652173913043478
Validation: 8
Validation accuracy: 0.9304347826086956
Validation: 9
Validation accuracy: 0.9652173913043478
Validation: 10
Validation accuracy: 0.9652173913043478

Test accuracy: 0.9305555555555556


In [7]:
# edRVFL

n_node = 10 # num of nodes in hidden layer
n_layer = 2 # num of hidden layers
lam = 1 # regularization parameter, lambda
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

for i, kf_values in enumerate(kf.split(X_train, y_train)):
#     print(f'train: {train_index}, val: {val_index}')
    print('Validation: {}'.format(i + 1))
    train_index, val_index = kf_values
    X_val_train, X_val_test = X_train[train_index], X_train[val_index]
    y_val_train, y_val_test = y_train[train_index], y_train[val_index]
    model = EnsembleDeepRVFL(n_node, lam, w_range, b_range, n_layer)
    model.train(X_val_train, y_val_train, n_class)
    prediction = model.predict(X_val_test, False)
    acc = model.eval(X_val_test, y_val_test)
    print(f'Validation accuracy: {acc}')
    val_acc.append(acc)
    if acc >= max(val_acc):
        max_index = train_index

X_train, y_train = X_train[max_index], y_train[max_index]
model = EnsembleDeepRVFL(n_node, lam, w_range, b_range, n_layer)
model.train(X_train, y_train, n_class)
prediction = model.predict(X_test, False)
acc = model.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')

Validation: 1
Validation accuracy: 0.9310344827586207
Validation: 2
Validation accuracy: 0.9482758620689655
Validation: 3
Validation accuracy: 0.9478260869565217
Validation: 4
Validation accuracy: 0.9565217391304348
Validation: 5
Validation accuracy: 0.9217391304347826
Validation: 6
Validation accuracy: 0.9304347826086956
Validation: 7
Validation accuracy: 0.9652173913043478
Validation: 8
Validation accuracy: 0.9565217391304348
Validation: 9
Validation accuracy: 0.9652173913043478
Validation: 10
Validation accuracy: 0.9391304347826087

Test accuracy: 0.9236111111111112


In [3]:
# BRVFL

n_node = 10 # num of nodes in hidden layer
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

for i, kf_values in enumerate(kf.split(X_train, y_train)):
#     print(f'train: {train_index}, val: {val_index}')
    print('Validation: {}'.format(i + 1))
    train_index, val_index = kf_values
    X_val_train, X_val_test = X_train[train_index], X_train[val_index]
    y_val_train, y_val_test = y_train[train_index], y_train[val_index]
    brvfl = BRVFL(n_node, w_range, b_range)
    brvfl.train(X_val_train, y_val_train, n_class)
    prediction = brvfl.predict(X_val_test, True)
    acc = brvfl.eval(X_val_test, y_val_test)
    print(f'Validation accuracy: {acc}')
    val_acc.append(acc)
    if acc >= max(val_acc):
        max_index = train_index

X_train, y_train = X_train[max_index], y_train[max_index]
brvfl = BRVFL(n_node, w_range, b_range)
brvfl.train(X_train, y_train, n_class)
prediction = brvfl.predict(X_test, True)
acc = brvfl.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')

Validation: 1



Convergence after  17  iterations
Validation accuracy: 0.896551724137931
Validation: 2



Convergence after  16  iterations
Validation accuracy: 0.8706896551724138
Validation: 3



Convergence after  17  iterations
Validation accuracy: 0.9217391304347826
Validation: 4



Convergence after  16  iterations
Validation accuracy: 0.9652173913043478
Validation: 5


...


Convergence after  17  iterations
Validation accuracy: 0.9304347826086956
Validation: 8



Convergence after  17  iterations
Validation accuracy: 0.8782608695652174
Validation: 9



Convergence after  16  iterations
Validation accuracy: 0.9652173913043478
Validation: 10



Convergence after  17  iterations
Validation accuracy: 0.8869565217391304



Convergence after  14  iterations

Test accuracy: 0.9166666666666666


In [2]:
# BdRVFL

n_node = 10 # num of nodes in hidden layer
n_layer = 5 # num of hidden layers
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

for i, kf_values in enumerate(kf.split(X_train, y_train)):
#     print(f'train: {train_index}, val: {val_index}')
    print('Validation: {}'.format(i + 1))
    train_index, val_index = kf_values
    X_val_train, X_val_test = X_train[train_index], X_train[val_index]
    y_val_train, y_val_test = y_train[train_index], y_train[val_index]
    model = BayesianDeepRVFL(n_node, w_range, b_range, n_layer)
    model.train(X_val_train, y_val_train, n_class)
    prediction = model.predict(X_val_test, True)
    acc = model.eval(X_val_test, y_val_test)
    print(f'Validation accuracy: {acc}')
    val_acc.append(acc)
    if acc >= max(val_acc):
        max_index = train_index

X_train, y_train = X_train[max_index], y_train[max_index]
model = BayesianDeepRVFL(n_node, w_range, b_range, n_layer)
model.train(X_train, y_train, n_class)
prediction = model.predict(X_test, True)
acc = model.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')

Validation: 1



Convergence after  16  iterations
Validation accuracy: 0.9051724137931034
Validation: 2



Convergence after  16  iterations
Validation accuracy: 0.8793103448275862
Validation: 3



Convergence after  15  iterations
Validation accuracy: 0.9391304347826087
Validation: 4



Convergence after  16  iterations
Validation accuracy: 0.9565217391304348
Validation: 5


...


Convergence after  16  iterations
Validation accuracy: 0.9043478260869565
Validation: 8



Convergence after  16  iterations
Validation accuracy: 0.9130434782608695
Validation: 9



Convergence after  16  iterations
Validation accuracy: 0.9652173913043478
Validation: 10



Convergence after  16  iterations
Validation accuracy: 0.9043478260869565



Convergence after  16  iterations

Test accuracy: 0.8645833333333334


In [2]:
# BedRVFL

n_node = 10 # num of nodes in hidden layer
n_layer = 5 # num of hidden layers
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

# for i, kf_values in enumerate(kf.split(X_train, y_train)):
# #     print(f'train: {train_index}, val: {val_index}')
#     print('Validation: {}'.format(i + 1))
#     train_index, val_index = kf_values
#     X_val_train, X_val_test = X_train[train_index], X_train[val_index]
#     y_val_train, y_val_test = y_train[train_index], y_train[val_index]
#     model = BayesianEnsembleDeepRVFL(n_node, w_range, b_range, n_layer)
#     model.train(X_val_train, y_val_train, n_class)
#     prediction = model.predict(X_val_test, False)
#     acc = model.eval(X_val_test, y_val_test)
#     print(f'Validation accuracy: {acc}')
#     val_acc.append(acc)
#     if acc >= max(val_acc):
#         max_index = train_index

# X_train, y_train = X_train[max_index], y_train[max_index]
model = BayesianEnsembleDeepRVFL(n_node, w_range, b_range, n_layer)
model.train(X_train, y_train, n_class)
prediction = model.predict(X_test, False)
acc = model.eval(X_test, y_test)
print(f'\nTest accuracy: {acc}')


Convergence after  17  iterations



Convergence after  16  iterations



Convergence after  17  iterations



Convergence after  17  iterations



Convergence after  17  iterations

Test accuracy: 0.9201388888888888
